In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# خواندن دیتا ها
!unzip "/content/drive/MyDrive/Project/Dataset/train.zip" -d "/content/train"
!unzip "/content/drive/MyDrive/Project/Dataset/validation.zip" -d "/content/validation"

In [ ]:
import os
import shutil
import tensorflow as tf
from PIL import Image
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

In [ ]:
# مسیر استخراج فایل‌ها
train_dir = "/content/train/train"
validation_dir = "/content/validation/validation"

In [ ]:
# نمایش محتویات پوشه‌های train و validation
print("Train files:", os.listdir(train_dir)[:10])  # نمایش 10 فایل اول
print("Validation files:", os.listdir(validation_dir)[:10])

In [ ]:
def remove_night_images(directory):
    files = os.listdir(directory)
    for file in files:
        if file.startswith('night'):
            file_path = os.path.join(directory, file)
            os.remove(file_path)  # حذف فایل

# حذف تصاویر شب از پوشه‌های train و validation
remove_night_images(train_dir)
remove_night_images(validation_dir)

# بررسی مجدد پوشه‌ها پس از حذف تصاویر شب
print("Remaining train files:", os.listdir(train_dir)[:10])
print("Remaining validation files:", os.listdir(validation_dir)[:10])


In [ ]:
def move_images_to_class_dir(source_dir, target_dir, class_name='car_plate'):
    class_dir = os.path.join(target_dir, class_name)

    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    for filename in os.listdir(source_dir):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            shutil.move(os.path.join(source_dir, filename), os.path.join(class_dir, filename))

# انتقال تصاویر به پوشه دسته‌بندی فرضی
move_images_to_class_dir(train_dir, train_dir)
move_images_to_class_dir(validation_dir, validation_dir)


In [ ]:
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    objects = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        objects.append({
            'name': name,
            'bbox': [xmin, ymin, xmax, ymax],
            'class_label': 1  # Assuming 'car_plate' is class 1
        })

    return root.find('filename').text, objects

In [ ]:
def load_data(image_dir, xml_dir):
    data = []

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)

            # Extract filename from xml_file (without extension)
            filename_without_ext = os.path.splitext(xml_file)[0]

            # Construct image filename with .jpg extension
            image_filename = filename_without_ext + '.jpg'

            # Construct the image path
            image_path = os.path.join(image_dir, image_filename)

            # Parse XML file
            _, objects = parse_xml(xml_path)

            # Append data (include bounding boxes and class labels)
            data.append({
                'image_path': image_path,
                'bboxes': [obj['bbox'] for obj in objects],
                'class_labels': [obj['class_label'] for obj in objects]
            })

    image_paths = [d['image_path'] for d in data]
    bboxes = [d['bboxes'] for d in data]
    class_labels = [d['class_labels'] for d in data]

    return image_paths, bboxes, class_labels

In [ ]:
train_images, train_bboxes, train_class_labels = load_data('/content/train/train/car_plate', '/content/train/train')
validation_images, validation_bboxes, validation_class_labels = load_data('/content/validation/validation/car_plate', '/content/validation/validation')

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load an Image for Data Augmentation Test
img_path = os.path.join(train_dir, train_images[0])
img = load_img(img_path, target_size=(512, 512))  # Use load_img directly

# Convert Image to Array
x = img_to_array(img)  # Use img_to_array directly
x = x.reshape((1,) + x.shape)

# Generate and Display Augmented Images
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(array_to_img(batch[0])) # Use array_to_img directly
    plt.show()

    i += 1
    if i > 5:
        break

In [ ]:
def resize_images(directory, target_size=(512, 512)):
    files = os.listdir(directory)
    for file in files:
        if file.endswith('.jpg'):  # Assuming all images are in JPG format
            img_path = os.path.join(directory, file)
            img = Image.open(img_path)
            img = img.resize(target_size)
            img.save(img_path)

# تغییر اندازه تصاویر train و validation به اندازه  512 x 512
resize_images('/content/train/train/car_plate', target_size=(512, 512))
resize_images('/content/validation/validation/car_plate', target_size=(512, 512))

print("Images resized to 512x512.")


In [ ]:
def create_model():
    model = Sequential()
    model.add(keras.Input(shape=(512, 512, 3)))

    # Block 1
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Block 2
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Block 3
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Block 4 (optional, you can add more blocks for deeper features)
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))

    # Classification Head
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Create an instance of the model
model = create_model()
model.summary()

In [ ]:
# تعریف EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
# تعریف ReduceLROnPlateau برای کاهش نرخ یادگیری
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    min_lr=1e-6
)

In [ ]:
# تنظیم seed برای تکرارپذیری نتایج
seed = 42

# تعریف داده‌افزایی
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

tf.random.set_seed(seed)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(512, 512),
    batch_size=64,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(512, 512),
    batch_size=64,
    class_mode='binary'
)

In [ ]:
def plot_augmented_images(generator, num_images=9, save=False):
    # گرفتن یک دسته از تصاویر و برچسب‌ها از ژنراتور
    x_batch, y_batch = next(generator)

    # بررسی ابعاد تصویر و تبدیل در صورت نیاز
    if np.max(x_batch) > 1:  # اگر داده‌ها در محدوده‌ی [0, 255] باشند، آن‌ها را به [0, 1] تبدیل می‌کنیم
        x_batch = x_batch / 255.0

    # نمایش تصاویر
    fig, axes = plt.subplots(3, 3, figsize=(10, 10))
    for i in range(num_images):
        ax = axes.flat[i]
        ax.imshow(np.clip(x_batch[i], 0, 1))  # اطمینان از اینکه داده‌ها در محدوده [0, 1] هستند
        ax.axis('off')

    plt.tight_layout()

    # ذخیره کردن تصویر در صورت نیاز
    if save:
        plt.savefig('augmented_images.png')

    plt.show()

# نمایش تصاویر تقویت شده با ذخیره کردن
plot_augmented_images(train_generator, save=True)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=len(train_images) // 32,
    validation_steps=len(validation_images) // 32
)

In [ ]:
# رسم نمودار از دست دادن (Loss)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
def plot_image_with_bbox(image, bbox, title="Prediction"):
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    # Check if bbox has 4 elements for (xmin, ymin, xmax, ymax)
    if len(bbox) == 4:
        xmin, ymin, xmax, ymax = bbox
    # If bbox is a list containing a single list/tuple with 2 elements
    elif len(bbox) == 1 and len(bbox[0]) == 4:
        xmin, ymin, xmax, ymax = bbox[0]  # Access the inner list/tuple
    # If bbox has 2 elements, assuming (x_center, y_center, width, height)
    # This part might need to be adjusted based on your actual bbox format
    #elif len(bbox) == 2: # Commenting to see if it addresses the error
    #    # Assumed structure (x_center, y_center, width, height)
    #    x_center, y_center, width, height = bbox[0] # Access the inner list/tuple
    #    xmin = x_center - (width / 2)
    #    ymin = y_center - (height / 2)
    #    xmax = x_center + (width / 2)
    #    ymax = y_center + (height / 2)
    else:
        raise ValueError(f"bbox has unexpected format: {bbox}, should have either 4 elements or be a list with a single 4-element list/tuple.")

    rect = plt.Rectangle((xmin * 224, ymin * 224), (xmax - xmin) * 224, (ymax - ymin) * 224,
                         linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    plt.title(title)
    plt.show()

In [ ]:
for i in range(5):
    img_path = validation_images[i]
    true_bbox = validation_class_labels[i][0]

    # Load the image with the size expected by the model
    img = image.load_img(img_path, target_size=(512, 512, 3))
    # Convert image to array
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.  # Normalize the image

    predicted_bbox = model.predict(img)[0]

    print(f"True BBox: {true_bbox}")
    print(f"Predicted BBox: {predicted_bbox}")

    # Display the original image (not preprocessed) in plot_image_with_bbox
    original_img = image.load_img(img_path, target_size=(512, 512))
    original_img = image.img_to_array(original_img) / 255.

    plot_image_with_bbox(original_img, true_bbox,model, title="True BBox")
    plot_image_with_bbox(original_img, predicted_bbox, model title="Predicted BBox")

In [ ]:
val_loss = model.evaluate(validation_images, validation_labels)
print(f"Validation Loss: {val_loss}")